In [94]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [95]:
import pandas as pd

In [96]:
df1 = pd.read_csv('data_frame2.txt')
df2 = pd.read_csv('data_frame3.txt')

In [98]:
df = df1.append(df2)

In [100]:
df['salary_type'] = [x.split()[1] for x in df['Salary Estimate']]

In [101]:
df['Salary Estimate'] = [x.split()[0] for x in df['Salary Estimate']]

In [102]:
print(df['salary_type'].unique())

['(Glassdoor' 'Per' '(Employer']


In [103]:
df['per_hour'] = [1 if 'Per' in x else 0 for x in df['salary_type']]
df['Employer_reported'] = [1 if 'Employer' in x else 0 for x in df['salary_type']]

In [104]:
#extracting avg salary from slaray estimation
df['avg_salary'] = (df['Salary Estimate'].map(lambda x: x.split('-')[0].lstrip('$').rstrip('k')).astype('int32')
                    +df['Salary Estimate'].map(lambda x: x.split('-')[1].lstrip('$').rstrip('k')).astype('int32'))/2

In [105]:
df.loc[df['Rating'] == -1, 'Company Name']
# it seems that the companies without rating have only their name in the Company Name column

5                    NanoMosaic
105                  Kronos Bio
144                 LGC Limited
165          Remix Therapeutics
168     Monte Rosa Therapeutics
                 ...           
227                 LGC Limited
230                       PRPDG
255    Well Established Biotech
296     Monte Rosa Therapeutics
297     Monte Rosa Therapeutics
Name: Company Name, Length: 61, dtype: object

In [59]:
df['Company Name'] = df.apply(lambda x: x['Company Name'] if x['Rating'] == -1 else x['Company Name'].split("\n")[0] , axis = 1)

In [60]:
df.drop(['Headquarters','Competitors','Salary Estimate'], axis = 1 , inplace = True)

In [64]:
# Since in Location columns there are instacnes wuth differnt structre, such as 'Virgina', we need to replace them with 
# the same structure as others so that we can extract usefull info from them

# We remove the ones without specified location (United States)
df = df[df['Location'] != 'United States']
list_1 = []
for item in df.Location:
    if ',' not in item:
        list_1.append(item)

set(list_1)

{'Colorado', 'Georgia', 'Missouri', 'New Jersey', 'Remote', 'Utah', 'Virginia'}

In [63]:
df['state'] = df.apply(lambda x: x['Location'] if ',' not in x['Location']
                       else x['Location'].split(',')[1].lstrip() , axis = 1)
# we create a dict to use for mapping
state_convertor = {'Colorado' : 'CO' , 'Georgia' : 'GA' , 'Missouri' : 'MO' , 'New Jersey' : 'NJ' ,
             'Remote' : 'Remote' , 'Utah' : 'UT' , 'Virginia' : 'VA'}
df['state'] = df['state'].map(lambda x: state_convertor[x] if x in state_convertor else x)

CA               239
MA                82
VA                72
MD                67
NY                59
TX                44
IL                31
Remote            30
WA                28
FL                25
PA                23
TN                21
MO                21
DC                21
MN                20
CO                19
NC                19
GA                16
United States     16
WI                16
NJ                15
UT                15
OH                15
AZ                15
MI                13
CT                 9
IN                 7
KS                 7
AL                 5
Arapahoe           4
DE                 2
OR                 2
Georgia            2
Virginia           2
RI                 2
NM                 2
NV                 1
Missouri           1
NE                 1
Utah               1
MS                 1
AR                 1
New Jersey         1
PR                 1
ID                 1
NH                 1
Colorado           1
KY           

In [83]:
# we are going to work on the size
print(df.Size.value_counts())
# Lets drop those company with unknown or -1 size
df = df[(df['Size'] != '-1') & (df['Size'] != 'Unknown')]
# Lets make a dictionary to convert differnt types of size to numbers, since they are compareable
size_convertor = {'1 to 50 Employees':1 , '51 to 200 Employees':2 , '201 to 500 Employees':3 , 
                 '501 to 1000 Employees':4 , '1001 to 5000 Employees':5 , '5001 to 10000 Employees':6 ,
                 '10000+ Employees':7}
df['Size'] = df['Size'].map(size_convertor)
print(df.Size.value_counts())

10000+ Employees           239
51 to 200 Employees        168
1001 to 5000 Employees     156
201 to 500 Employees       127
501 to 1000 Employees      121
1 to 50 Employees           81
5001 to 10000 Employees     61
Name: Size, dtype: int64
7    239
2    168
5    156
3    127
4    121
1     81
6     61
Name: Size, dtype: int64


In [85]:
# Now we want to change the founded year to age of the company
df['Founded'] = 2020 - df['Founded']

In [87]:
df.Industry.value_counts()

Biotech & Pharmaceuticals                  211
Health Care Services & Hospitals           106
Computer Hardware & Software                66
Internet                                    57
Enterprise Software & Network Solutions     56
                                          ... 
Grantmaking Foundations                      1
Music Production & Distribution              1
Farm Support Services                        1
Home Centers & Hardware Stores               1
Industrial Manufacturing                     1
Name: Industry, Length: 73, dtype: int64

In [88]:
df.Sector.value_counts()

Information Technology                231
Biotech & Pharmaceuticals             211
Business Services                     119
Health Care                           106
Manufacturing                          39
Finance                                35
-1                                     32
Insurance                              32
Aerospace & Defense                    29
Retail                                 24
Media                                  22
Government                             17
Education                              16
Oil, Gas, Energy & Utilities           11
Telecommunications                      8
Transportation & Logistics              6
Accounting & Legal                      4
Restaurants, Bars & Food Services       4
Non-Profit                              2
Agriculture & Forestry                  2
Consumer Services                       2
Construction, Repair & Maintenance      1
Name: Sector, dtype: int64

In [91]:
df2 = pd.read_csv('data_frame3.txt')

In [99]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$57k-$102k (Glassdoor Est.),A trusted partner at nearly three million comm...,3.6,Ecolab\n3.6,"Saint Paul, MN",-1,10000+ Employees,1923,Company - Public,Chemical Manufacturing,Manufacturing,$10+ billion (USD),-1
1,1,Data Scientist,$57k-$102k (Glassdoor Est.),As Data Scientist in the Pricing Analytics tea...,4.1,HP Inc.\n4.1,"San Diego, CA",-1,10000+ Employees,1939,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
2,2,Data Scientist,$57k-$102k (Glassdoor Est.),Job Brief\n\nThe ideal candidate will have pre...,2.9,IFG Companies\n2.9,"Atlanta, GA",-1,201 to 500 Employees,1985,Company - Private,Insurance Carriers,Insurance,Unknown / Non-Applicable,-1
3,3,Data Scientist,$57k-$102k (Glassdoor Est.),Every Analyst a Scientist - One of our primary...,4.5,Systems & Technology Research\n4.5,"Woburn, MA",-1,201 to 500 Employees,2010,Company - Private,Aerospace & Defense,Aerospace & Defense,$100 to $500 million (USD),-1
4,4,Senior Data Scientist,$57k-$102k (Glassdoor Est.),"About Us\n\nAt GutCheck, we pioneered agile ma...",3.9,GutCheck\n3.9,"Denver, CO",-1,51 to 200 Employees,2009,Company - Private,Research & Development,Business Services,$10 to $25 million (USD),-1
